In [5]:
# -*- coding: utf-8 -*-
# Copyright (C) 2023 # @version
# @Time    : 2023/5/20
# @Author  : Oraer 和 Kibety
# @File    : GoldMiner.ipynb

import cv2
import numpy as np
import pyautogui
import time

class GoldMiner:
    def __init__(self, 
                 gold_pth = None, 
                 diamond_pth = None, 
                 bag_pth = None, 
                 stone_pth = None):
        # 加载素材包
        self.gold_pth = gold_pth
        self.target_list = []
        self.diamond_pth = diamond_pth
        self.diamond_list = []
        self.bag_pth = bag_pth
        self.bag_list = []
        self.stone_pth = stone_pth
        self.stone_list = []
        # 基础配置
        self.screenshot = None
        self.screenshot_gray = None
        self.line_location = None

    def set_screenshot(self, screenshot_pth):
        """
        获取截图
        """
        # self.screenshot = cv2.imread(screenshot_pth)
        self.screenshot = screenshot_pth
        height, width, channel = self.screenshot.shape
        # 重置分辨率 3050/2 1920/2   |  1920/1.5 1080/1.5
        self.screenshot = cv2.resize(self.screenshot, (int(1920/1.5), int(1080/1.5)), interpolation=cv2.INTER_CUBIC)
        self.screenshot_gray = cv2.cvtColor(self.screenshot, cv2.COLOR_BGR2GRAY)

    def least_squares_fit(self, lines):
        """
        将lines中的线段拟合成一条线段
        :param lines: 线段集合，[np.array([[x_1 ,y_1 ,x_2 ,y_2]]), np.array([[x_1, y_1, x_2, y_2])
        :return: 线段上的两点, np.arrray([[x_min, y_min], [x_max, y_max]])
        """
        # 1. 获取坐标点
        x_coords = np.ravel([[line[0][0], line[0][2]] for line in lines])
        y_coords = np.ravel([[line[0][1], line[0][3]] for line in lines])
        # 2. 进行直线拟合， 得到多项式系数 deg=1 表示拟合的是一次多项式函数，即一条直线。
        poly = np.polyfit(x_coords, y_coords, deg=1)

        # 3. 根据多项式系数，计算直线上的两个端点， 用于唯一确定这条直线
        point_min = (np.min(x_coords), np.polyval(poly, np.min(x_coords)))
        point_max = (np.max(x_coords), np.polyval(poly, np.max(x_coords)))

        return np.array([point_min, point_max], dtype=np.int64)
    
    def get_line(self):
        """
        获取钩子所在直线
        """
        # 旋转点坐标 先列后行 0.502 0.0857  | 0.503 0.095
        anchor_point = [int(0.5075*self.screenshot.shape[1]), int(0.101*self.screenshot.shape[0])]

        # 裁剪方框，定位左上角,右下角坐标位置
        # 0.111 0.475 0.145 0.53 | 0.108 0.465 0.146 0.53
        cut_box_location = ((int(0.1167*self.screenshot.shape[0]), int(0.465*self.screenshot.shape[1])),
                            (int(0.155*self.screenshot.shape[0]), int(0.566*self.screenshot.shape[1])))
        cropped_image = self.screenshot[cut_box_location[0][0]:cut_box_location[1][0], cut_box_location[0][1]:cut_box_location[1][1]]
        cropped_image = cv2.resize(cropped_image, (int(10 * cropped_image.shape[1]), int(10 * cropped_image.shape[0])), interpolation=cv2.INTER_NEAREST)

        # debug crop
        # cv2.imshow("crop", cropped_image)
        # cv2.waitKey(0)

        # debug anchor_point
        # cv2.circle(self.screenshot, anchor_point, 3, (255, 255, 0), thickness=-1)
        # cv2.imshow("screen", self.screenshot)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

        # 将彩色图片转换为灰度图片
        gray_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
        gray_image[gray_image > 61] = 0

        # debug gray
        # cv2.imshow("gray_img", gray_image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

        # 创建一个LSD对象
        lsd = cv2.createLineSegmentDetector(0)
        # 执行检测结果
        dlines = lsd.detect(gray_image)

        # 直线拟合
        if dlines[0] is None:
            return None
        line = self.least_squares_fit(dlines[0])

        # 坐标转换
        line_tran = ((cut_box_location[0][1]+int(line[0][0]/10), (cut_box_location[0][0]+int(line[0][1]/10))),
                    (cut_box_location[0][1]+int(line[1][0]/10), (cut_box_location[0][0]+int(line[1][1]/10))))

        # 中点计算
        midpoint = (int(abs(line_tran[0][0] + line_tran[1][0])/2), int(abs(line_tran[0][1] + line_tran[1][1])/2))

        # 结果可视化
        # 绘制拟合中点与旋转点
        for point in [anchor_point, midpoint]:
            x, y = point
            xy = "[%d,%d]" % (x, y)
            cv2.circle(self.screenshot, (x, y), 3, (255, 0, 0), thickness=1)
            cv2.putText(self.screenshot, xy, (x, y + 10), cv2.FONT_HERSHEY_PLAIN,
                        1.0, (107, 73, 251), thickness=1)  # 如果不想在图片上显示坐标可以注释该行
        # 绘制拟合直线
        cv2.line(self.screenshot, midpoint, anchor_point,
                    color=(0, 255, 255), thickness=1)

        self.line_location = np.array([anchor_point, midpoint])

    def get_result(self, label, hsv_range):
        """
        获取目标信息
        """
        height, width, channel = self.screenshot.shape
        crop_position = [0, int(0.15 * height)]
        cropped = self.screenshot[int(0.15*height):, :]

        hsv_img = cv2.cvtColor(cropped, cv2.COLOR_BGR2HSV)
        # 低阈值和高阈值
        low_hsv = np.array(hsv_range[0])
        high_hsv = np.array(hsv_range[1])
        mask = cv2.inRange(hsv_img, low_hsv, high_hsv)
        # 得到掩模后，进行形态学操作 开运算 闭运算，去除噪点
        kernel = np.ones((3, 3), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        # 找到掩模上白色物体的轮廓
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        objects = []

        # 简化代码
        # objects_append = objects.append
        # xywh = [cv2.boundingRect(contour) for contour in contours]
        # # plot = [cv2.rectangle(cropped, (x, y), (x + w, y + h), (0, 255, 0), 1) for x, y, w, h in xywh]
        # objects = [[x + crop_position[0] + int(0.5*w), y + crop_position[1] + int(0.5*h)] for x, y, w, h in xywh]

        if len(contours) == 0:
            print("未检测到目标{}".format(label))
        else:
            print("检测到目标{}".format(label))
            for contour in contours:
                x, y, w, h = cv2.boundingRect(contour)
                # 位置矫正
                x = x + crop_position[0]
                y = y + crop_position[1]
                cv2.rectangle(self.screenshot, (x, y), (x + w, y + h), (0, 255, 0), 1)
                # cv2.putText(self.screenshot, label, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
                # 中心点位置
                center_x, center_y = int(x + 0.5 * w), int(y + 0.5 * h)
                # print("中心位置：", center_x, center_y)
                objects.append([center_x, center_y])
 
        return np.array(objects)


    def get_gold(self):
        """
        获取黄金信息
        """
        label = 'gold'
        hsv_range = [[22, 230, 160], [31, 255, 255]]
        self.target_list.append(self.get_result(label, hsv_range))
            
        
    
    def get_diamond(self):
        """
        获取钻石信息
        """
        label = 'diamond'
        hsv_range = [[87, 30, 218], [110, 130, 240]]
        self.target_list.append(self.get_result(label, hsv_range))

    def get_bag(self):
        """
        获取袋子信息
        [[10, 220, 220], [30, 255, 255]]
        [[10, 200, 240], [22, 255, 255]]
        """
        label = 'bag'
        hsv_range =  [[10, 200, 240], [22, 255, 255]]
        self.target_list.append(self.get_result(label, hsv_range))

    def hook_control(self):
        """
        # 计算绳索与目标中心点角度值
        # 遍历(大)黄金中心点坐标列表，并计算角度值
        self.line_location: [[旋转点的坐标], [钩子线中点坐标]] 
        self.target_list[0]: 黄金中心点坐标
        self.target_list[1]: 钻石中心点坐标
        """
        # 拟合旋转点到绳索的直线
        vx1, vy1 = cv2.fitLine(self.line_location, cv2.DIST_L2, 0, 0.01, 0.01)[:2].flatten()   
        # print("self.line_location :", self.line_location)
        # print("黄金中心点坐标列表：", self.target_list)
        for targets in self.target_list:
            for big_point in targets:
                each_line = np.array([self.line_location[0], big_point])

                # 拟合旋转点到黄金的直线
                vx2, vy2 = cv2.fitLine(each_line, cv2.DIST_L2, 0, 0.01, 0.01)[:2].flatten()        
                v1 = np.array([vx1, vy1])   # 向量1 旋转点到绳索
                v2 = np.array([vx2, vy2])   # 向量2 旋转点到黄金

                # 计算向量长度
                norm_v1 = np.linalg.norm(v1)
                norm_v2 = np.linalg.norm(v2)

                # 检查向量是否为零向量
                if norm_v1 == 0 or norm_v2 == 0:
                    angle = np.nan
                else:
                    # 计算向量之间的夹角
                    angle = np.arccos(np.dot(v1, v2) / (norm_v1 * norm_v2))
                # angle = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
                angle_degrees = np.degrees(angle)
                
                # print("angle_degrees", angle_degrees)
                
                # 角度小于5时
                if angle_degrees < 4:
                    cv2.line(self.screenshot, tuple(each_line[0]), tuple(each_line[1]), (0, 0, 255), 1)
                    cv2.putText(self.screenshot, f"Angle: {angle_degrees:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1)
                    pyautogui.keyDown('down')
                    pyautogui.keyUp('down')

                
    def show_result_image(self):
        """
        可视化识别结果 
        """        
        return self.screenshot

In [6]:
if __name__ == '__main__':
    gold_miner = GoldMiner()
    # screenshot = wincap.get_screenshot()
    screenshot = cv2.imread('./images/pm_9.png')
    height, width, channel = screenshot.shape
    # screenshot 补充为 1920*1080
    padded_img = np.pad(screenshot, ((1080-height, 0), (1920-width, 0), (0, 0)), mode='constant')
    padded_img = padded_img[:1080, :1920, :]
    # 设置屏幕截图
    gold_miner.set_screenshot(screenshot_pth = padded_img)
    # 计算绳子参数
    gold_miner.get_line()
    # 获取黄金信息
    gold_miner.get_gold()
    # 获取钻石信息
    # gold_miner.get_diamond()
    # 获取袋子信息
    # gold_miner.get_bag()
    # 控制钩子
    gold_miner.hook_control()
    
    result =  gold_miner.show_result_image()

    # 可视化结果
    # cv2.imshow("Result", result)
    # cv2.waitKey()
    # cv2.destroyAllWindows()  

    # # 按键检测
    # pressedKey = cv2.waitKey(1) & 0xFF
    # if pressedKey == ord('q'):
    #     cv2.destroyAllWindows()
    #     print("退出程序")

检测到目标gold


In [7]:
import numpy as np
import time
from toolkit import WindowCapture


if __name__ == '__main__':
    # Gold Miner 🕹️ Play Gold Miner on CrazyGames - 360极速浏览器 13.5
    wincap = WindowCapture("Gold Miner 🕹️ Play Gold Miner on CrazyGames - 360极速浏览器 13.5")
    hwnd = wincap.hwnd

    # 等待屏幕焦点
    print("等待屏幕焦点。。。。")
    time.sleep(1)

    loop_time = time.time()
    fps = "waiting..."
    # 实例化
    gold_miner = GoldMiner(gold_pth = './gold/1.png', diamond_pth = './diamond/1.png')
    is_gaming_window = True
    while True:
        # 平均每次截图需要0.033秒
        screenshot = wincap.get_screenshot()
        
        # screenshot 补充为 1920*1080
        height, width, channel = screenshot.shape
        padded_img = np.pad(screenshot, ((1080-height, 0), (1920-width, 0), (0, 0)), mode='constant')
        padded_img = padded_img[:1080, :1920, :]
        
        # 设置屏幕截图
        gold_miner.set_screenshot(screenshot_pth = padded_img)

        # 判断是否为游戏窗口
        # height, width, channel = screenshot.shape
        point = [int(0.075*height),int(0.465*width)]
        a = np.array(screenshot[point[0], point[1]])
        b = np.array([155, 64, 40])

        if np.array_equal(a, b):
            
            is_gaming_window = True

            # 清空目标列表
            gold_miner.target_list.clear()
            # 获取钻石信息
            gold_miner.get_diamond()
            # 获取袋子信息
            gold_miner.get_bag()
            # 获取黄金信息
            gold_miner.get_gold()
            # 计算绳子参数
            gold_miner.get_line()
            # 控制钩子
            gold_miner.hook_control()
        else:
            is_gaming_window = False
            print("非游戏窗口...")
             
        # 游戏窗口可视化
        fps = 'FPS: {}'.format(int(1 / (time.time() - loop_time)))
        result = gold_miner.show_result_image()
        cv2.putText(result, fps, (35, 60), cv2.FONT_HERSHEY_PLAIN,
                    1.5, (107, 73, 251), thickness=2)                   # 如果不想在图片上显示FPS可以注释该行
        cv2.imshow('Game Vision', result)
        loop_time = time.time()

        # 按键检测
        pressedKey = cv2.waitKey(1) & 0xFF
        if pressedKey == ord('s'):
            cv2.imwrite("testimg.png", screenshot)
        elif pressedKey == ord('q'):
            cv2.destroyAllWindows()
            print("退出程序")
            break

    print('Done...')


等待屏幕焦点。。。。
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
非游戏窗口...
未检测到目标diamond
检测到目标bag
检测到目标gold
未检测到目标diamond
检测到目标bag
检测到目标gold
未检测到目标diamond
检测到目标bag
检测到目标gold
未检测到目标di